In [18]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

In [1]:
!pip install yacs
!pip install scipy
!pip install tqdm
!pip install yacs
!pip install Cython
!pip install matplotlib>=3.2.2
!pip install numpy>=1.18.5
!pip install opencv-python>=4.1.2
!pip install Pillow
!pip install PyYAML>=5.3
!pip install scipy>=1.4.1
!pip install tensorboardX
!pip install seaborn
!pip install prefetch_generator
!pip install imageio
!pip install scikit-learn

   ---------------------------------------- 0.0/144.7 kB ? eta -:--:--
   -------- ------------------------------- 30.7/144.7 kB ? eta -:--:--
   --------------------------------- ------ 122.9/144.7 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------  143.4/144.7 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 144.7/144.7 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ----------------------------------- ---- 51.2/57.6 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 57.6/57.6 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.3 kB ? eta -:--:--
   ------------------------------------ --- 71.7/78.3 kB 3.8 MB/s eta 0:00:01
   ---------------------------------------- 78.3/78.3 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.8 MB 3.6 MB/s eta 0:00:01
   -- --------

(optional) Exporting a Model from PyTorch to ONNX and Running it using ONNX Runtime
===================================================================================

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>As of PyTorch 2.1, there are two versions of ONNX Exporter.</p>
</div>

In this tutorial, we describe how to convert a model defined in PyTorch
into the ONNX format using the TorchScript
[\`torch.onnx.export]{.title-ref} ONNX exporter.

The exported model will be executed with ONNX Runtime. ONNX Runtime is a
performance-focused engine for ONNX models, which inferences efficiently
across multiple platforms and hardware (Windows, Linux, and Mac and on
both CPUs and GPUs). ONNX Runtime has proved to considerably increase
performance over multiple models as explained
[here](https://cloudblogs.microsoft.com/opensource/2019/05/22/onnx-runtime-machine-learning-inferencing-0-4-release)

For this tutorial, you will need to install
[ONNX](https://github.com/onnx/onnx) and [ONNX
Runtime](https://github.com/microsoft/onnxruntime). You can get binary
builds of ONNX and ONNX Runtime with

``` {.sourceCode .bash}
%%bash
pip install onnx onnxruntime
```

ONNX Runtime recommends using the latest stable runtime for PyTorch.


In [19]:
%pip install onnx onnxruntime

In [2]:
# Some standard imports
import numpy as np

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

Super-resolution is a way of increasing the resolution of images, videos
and is widely used in image processing or video editing. For this
tutorial, we will use a small super-resolution model.

First, let\'s create a `SuperResolution` model in PyTorch. This model
uses the efficient sub-pixel convolution layer described in [\"Real-Time
Single Image and Video Super-Resolution Using an Efficient Sub-Pixel
Convolutional Neural Network\" - Shi et
al](https://arxiv.org/abs/1609.05158) for increasing the resolution of
an image by an upscale factor. The model expects the Y component of the
`YCbCr` of an image as an input, and outputs the upscaled Y component in
super resolution.

[The
model](https://github.com/pytorch/examples/blob/master/super_resolution/model.py)
comes directly from PyTorch\'s examples without modification:


In [3]:
# Create the yolop model by using the above model definition.
torch_model = torch.hub.load('hustvl/yolop', 'yolop', pretrained=True)

c:\Users\Berne\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\hub.py:293: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/hustvl/yolop/zipball/main" to C:\Users\Berne/.cache\torch\hub\main.zip


Ordinarily, you would now train this model; however, for this tutorial,
we will instead download some pretrained weights. Note that this model
was not trained fully for good accuracy and is used here for
demonstration purposes only.

It is important to call `torch_model.eval()` or
`torch_model.train(False)` before exporting the model, to turn the model
to inference mode. This is required since operators like dropout or
batchnorm behave differently in inference and training mode.


Exporting a model in PyTorch works via tracing or scripting. This
tutorial will use as an example a model exported by tracing. To export a
model, we call the `torch.onnx.export()` function. This will execute the
model, recording a trace of what operators are used to compute the
outputs. Because `export` runs the model, we need to provide an input
tensor `x`. The values in this can be random as long as it is the right
type and size. Note that the input size will be fixed in the exported
ONNX graph for all the input\'s dimensions, unless specified as a
dynamic axes. In this example we export the model with an input of
batch\_size 1, but then specify the first dimension as dynamic in the
`dynamic_axes` parameter in `torch.onnx.export()`. The exported model
will thus accept inputs of size \[batch\_size, 1, 224, 224\] where
batch\_size can be variable.

To learn more details about PyTorch\'s export interface, check out the
[torch.onnx documentation](https://pytorch.org/docs/master/onnx.html).


In [5]:
# Input to the model
img = torch.randn(1,3,640,640)
torch_out = torch_model(img)

# Export the model
torch.onnx.export(torch_model,               # model being run
                  img,                         # model input (or a tuple for multiple inputs)
                  "yolop.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

C:\Users\Berne/.cache\torch\hub\hustvl_yolop_main\lib\models\common.py:199: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.grid[i].shape[2:4] != x[i].shape[2:4]:


SymbolicValueError: Unsupported: ONNX export of index_put in opset 9. Please try opset version 11.  [Caused by the value '1113 defined in (%1113 : Float(*, 3, 80, 80, 6, strides=[115200, 38400, 480, 6, 1], requires_grad=1, device=cpu) = onnx::Sigmoid(%1110), scope: lib.models.YOLOP.MCnet::/lib.models.common.Detect::model.24 # C:\Users\Berne/.cache\torch\hub\hustvl_yolop_main\lib\models\common.py:201:0
)' (type 'Tensor') in the TorchScript graph. The containing node has kind 'onnx::Sigmoid'.] 
    (node defined in C:\Users\Berne/.cache\torch\hub\hustvl_yolop_main\lib\models\common.py(201): forward
c:\Users\Berne\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py(1522): _slow_forward
c:\Users\Berne\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py(1541): _call_impl
c:\Users\Berne\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py(1532): _wrapped_call_impl
C:\Users\Berne/.cache\torch\hub

We also computed `torch_out`, the output after of the model, which we
will use to verify that the model we exported computes the same values
when run in ONNX Runtime.

But before verifying the model\'s output with ONNX Runtime, we will
check the ONNX model with ONNX API. First,
`onnx.load("super_resolution.onnx")` will load the saved model and will
output a `onnx.ModelProto` structure (a top-level file/container format
for bundling a ML model. For more information [onnx.proto
documentation](https://github.com/onnx/onnx/blob/master/onnx/onnx.proto).).
Then, `onnx.checker.check_model(onnx_model)` will verify the model\'s
structure and confirm that the model has a valid schema. The validity of
the ONNX graph is verified by checking the model\'s version, the
graph\'s structure, as well as the nodes and their inputs and outputs.


In [24]:
import onnx

onnx_model = onnx.load("yolop.onnx")
onnx.checker.check_model(onnx_model)

Now let\'s compute the output using ONNX Runtime\'s Python APIs. This
part can normally be done in a separate process or on another machine,
but we will continue in the same process so that we can verify that ONNX
Runtime and PyTorch are computing the same value for the network.

In order to run the model with ONNX Runtime, we need to create an
inference session for the model with the chosen configuration parameters
(here we use the default config). Once the session is created, we
evaluate the model using the run() API. The output of this call is a
list containing the outputs of the model computed by ONNX Runtime.


In [25]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("yolop.onnx", providers=["CPUExecutionProvider"])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


We should see that the output of PyTorch and ONNX Runtime runs match
numerically with the given precision (`rtol=1e-03` and `atol=1e-05`). As
a side-note, if they do not match then there is an issue in the ONNX
exporter, so please contact us in that case.


Running the model on an image using ONNX Runtime
================================================


So far we have exported a model from PyTorch and shown how to load it
and run it in ONNX Runtime with a dummy tensor as an input.


For this tutorial, we will use a famous cat image used widely which
looks like below

![](https://pytorch.org/tutorials/_static/img/cat_224x224.jpg)


First, let\'s load the image, preprocess it using standard PIL python
library. Note that this preprocessing is the standard practice of
processing data for training/testing neural networks.

We first resize the image to fit the size of the model\'s input
(224x224). Then we split the image into its Y, Cb, and Cr components.
These components represent a grayscale image (Y), and the
blue-difference (Cb) and red-difference (Cr) chroma components. The Y
component being more sensitive to the human eye, we are interested in
this component which we will be transforming. After extracting the Y
component, we convert it to a tensor which will be the input of our
model.


In [26]:
! wget "https://pytorch.org/tutorials/_static/img/cat_224x224.jpg" -O cat.jpg
# ! wget 'https://docs.google.com/uc?export=download&id=1emp4FIwheYnCGm1klhMVeAUxt9qAURoZ' -O alot5classes.tar

--2024-05-03 11:59:13--  https://pytorch.org/tutorials/_static/img/cat_224x224.jpg
Resolving pytorch.org (pytorch.org)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to pytorch.org (pytorch.org)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10270 (10K) [image/jpeg]
Saving to: ‘cat.jpg’

cat.jpg             100%[===================>]  10.03K  --.-KB/s    in 0s      

2024-05-03 11:59:14 (61.7 MB/s) - ‘cat.jpg’ saved [10270/10270]



In [27]:
from PIL import Image
import torchvision.transforms as transforms

img = Image.open("./cat.jpg")

resize = transforms.Resize([224, 224])
img = resize(img)

img_ycbcr = img.convert('YCbCr')
img_y, img_cb, img_cr = img_ycbcr.split()

to_tensor = transforms.ToTensor()
img_y = to_tensor(img_y)
img_y.unsqueeze_(0)

tensor([[[[0.2157, 0.1961, 0.1922,  ..., 0.5294, 0.5569, 0.5686],
          [0.2039, 0.1961, 0.1922,  ..., 0.5333, 0.5569, 0.5686],
          [0.1961, 0.1843, 0.1843,  ..., 0.5216, 0.5412, 0.5490],
          ...,
          [0.6667, 0.6745, 0.6392,  ..., 0.6902, 0.6667, 0.6078],
          [0.6392, 0.6431, 0.6235,  ..., 0.8000, 0.7608, 0.6745],
          [0.6392, 0.6353, 0.6510,  ..., 0.8118, 0.7686, 0.6667]]]])

Now, as a next step, let\'s take the tensor representing the grayscale
resized cat image and run the super-resolution model in ONNX Runtime as
explained previously.


In [28]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img_y)}
ort_outs = ort_session.run(None, ort_inputs)
img_out_y = ort_outs[0]

At this point, the output of the model is a tensor. Now, we\'ll process
the output of the model to construct back the final output image from
the output tensor, and save the image. The post-processing steps have
been adopted from PyTorch implementation of super-resolution model
[here](https://github.com/pytorch/examples/blob/master/super_resolution/super_resolve.py).


In [29]:
img_out_y = Image.fromarray(np.uint8((img_out_y[0] * 255.0).clip(0, 255)[0]), mode='L')

# get the output image follow post-processing step from PyTorch implementation
final_img = Image.merge(
    "YCbCr", [
        img_out_y,
        img_cb.resize(img_out_y.size, Image.BICUBIC),
        img_cr.resize(img_out_y.size, Image.BICUBIC),
    ]).convert("RGB")

# Save the image, we will compare this with the output image from mobile device
final_img.save("./cat_superres_with_ort.jpg")

![](https://pytorch.org/tutorials/_static/img/cat_superres_with_ort.jpg)

ONNX Runtime being a cross platform engine, you can run it across
multiple platforms and on both CPUs and GPUs.

ONNX Runtime can also be deployed to the cloud for model inferencing
using Azure Machine Learning Services. More information
[here](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-onnx).

More information about ONNX Runtime\'s performance
[here](https://github.com/microsoft/onnxruntime#high-performance).

For more information about ONNX Runtime
[here](https://github.com/microsoft/onnxruntime).
